### Basic Reminder: Covered More Heavily in Other Notebook

1) `deco` returns its `inner` function object

2) target is decorated by deco

3) Invoking `target` actually executes `inner`

4) `target` is just a reference to `inner` now

In [2]:
>>> def deco(func):
...     def inner():
...         print('running inner()')
...     return inner  # 1
...
>>> @deco
... def target():  #2
...     print('running target()')
...
>>> print(target())  #3

>>> print(target)  #4

running inner()
None
<function deco.<locals>.inner at 0x102ffbaf0>


### When Does Python Execute Decorators? 

- `register` runs twice before any other function. 
- The main point: function decorators are executed as soon as the module is imported, but the decorated functions only run when invoked. 

In [10]:
registry = []  

def register(func):  
    print(f'running register({func})')  # display what function is being decorated
    registry.append(func) # hold reference to functions decorated
    return func  

@register  
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3() # not decorated

running register(<function f1 at 0x102ffb0d0>)
running register(<function f2 at 0x102ffb430>)


In [11]:
main()
print(registry)

running main()
registry -> [<function f1 at 0x102ffb0d0>, <function f2 at 0x102ffb430>]
running f1()
running f2()
running f3()
[<function f1 at 0x102ffb0d0>, <function f2 at 0x102ffb430>]


### Variable Scoping

In [12]:
def f1(a):
    print(a)
    print(b) # this should error since we don't have a `b` anywhere
f1(3)

3


NameError: name 'b' is not defined

In [13]:
# now we define a global var b:
b = 6
f1(3)

3
6


In [17]:
# now a whacky example: 
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [19]:
from dis import dis

dis(f2) # Load fast means it is looking for local -> this is where we run into our issue

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [20]:
dis(f1) # We actually had LOAD GLOBAL for variable b, which is why it does not error 

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


#### What Is Happening Here?

Python seees a reference to the `b` in the function and assigns it as a local variable, which is tries to fetch from local scope. 

It finds that `b` is unbound. 

In [16]:
# now a different approach 
b = 6

def f3(a):
    global b # we have now stated there is a global b that we can use? I think? 
    print(a)
    print(b)
    b = 9
    
f3(3) # we will yield the 6

# but b is now a 9...
print(b)

3
6
9


### Onto Closures

This is a pretty interesting topic which begins with an example of having an object to hold the on-going average of a series. 


#### Using OOP

Pretty obvious what is happening here. 

In [4]:
class Averager():

    def __init__(self):
        self.series = [] # attribute to store off series 

    def __call__(self, new_value):
        self.series.append(new_value) # add to series list 
        total = sum(self.series) # find sum (not stored as attr)
        return total / len(self.series) # return avg (sum / n)

In [5]:
avg = Averager()

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


#### Higher-order function 

We now have a function object defined within `make_averager`. 

In [6]:
def make_averager():
    series = [] # stores a list, local to "make_averager"

    def averager(new_value):
        series.append(new_value) # references "free variable" "series"
        total = sum(series)
        return total / len(series)

    return averager


avg = make_averager()

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [8]:
# interestingly, this function has variables and free variables
print(avg.__code__.co_varnames) # variables in func
print(avg.__code__.co_freevars) # free variable

# we can even look at the closure itself
print(avg.__closure__)

# and get values
print(avg.__closure__[0].cell_contents)

('new_value', 'total')
('series',)
(<cell at 0x10820f070: list object at 0x108f05580>,)
[10, 11, 12]


### So, What Is A Closure? 

Really just functions with data attached. It is a function that references variables from a containing scope, like an inner function referencing variables declared only in an outer function.

- "A function that can refer to environments that are no longer active. A closure allows you to bind variables into a function without passing them as parameters"

### Nonlocal

The previous function was inefficient from a memory perspective as it required us to keep track of a large, memory intensive list object. 

Instead, we should just keep tabs on:
- a running `total` (the sum) 
- a `count` of items (just distinct count) 

But this yields an issue, shown below: 
- When we use the `var += 1` syntax, it is really `var = var + 1`
- This means we assign to `var` in the body of the function, which makes it a local variable...so we run into errors. 

In [10]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1 # interpreted as count = count + 1
        total += new_value # total = total + 1
        return total / count

    return averager


# try it out
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

The solution is to use the `nonlocal` keyword. This will allow us to "declare a variable as a free variables even when defined within the function"

In [11]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

# try it out
avg = make_averager()
avg(10)

10.0

### Implementing Simple Decorator: 

In [14]:
import time


def clock(func):
    def clocked(*args):  #1: Allows for any number of arguments
        t0 = time.perf_counter() # start timer
        result = func(*args)  #2: Decorated func is a free vars
        elapsed = time.perf_counter() - t0 # end timer
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked  # return inner func to replace decorated func

In [16]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12473192s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000121s] factorial(1) -> 1
[0.00005358s] factorial(2) -> 2
[0.00010129s] factorial(3) -> 6
[0.00014783s] factorial(4) -> 24
[0.00019467s] factorial(5) -> 120
[0.00024604s] factorial(6) -> 720
6! = 720


### memoization 

Used to speed up computer programs by storing results of expensive function calls and returning "cached" result when same inputs occur again.

`@cache` works by:
- taking hashable arguments and storing results in a dict 

#### LRI

Least Recently Used: Basically, older entries that haven't been read in awhile will be discarded to make room for new ones. 
- includes a `maxsize` parameter that sets max number of entries. 

In [18]:
# recursion: we will exxecute the same function call multiple times
# fib(1) -> 8 times
# fib(2) -> 5 times...
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)


if __name__ == '__main__':
    print(fibonacci(6)) 

[0.00000058s] fibonacci(0) -> 0
[0.00000054s] fibonacci(1) -> 1
[0.00006983s] fibonacci(2) -> 1
[0.00000033s] fibonacci(1) -> 1
[0.00000050s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00016354s] fibonacci(2) -> 1
[0.00020271s] fibonacci(3) -> 2
[0.00030858s] fibonacci(4) -> 3
[0.00000033s] fibonacci(1) -> 1
[0.00000037s] fibonacci(0) -> 0
[0.00000038s] fibonacci(1) -> 1
[0.00041483s] fibonacci(2) -> 1
[0.00045912s] fibonacci(3) -> 2
[0.00000042s] fibonacci(0) -> 0
[0.00000058s] fibonacci(1) -> 1
[0.00029292s] fibonacci(2) -> 1
[0.00000038s] fibonacci(1) -> 1
[0.00000037s] fibonacci(0) -> 0
[0.00000042s] fibonacci(1) -> 1
[0.00003475s] fibonacci(2) -> 1
[0.00006921s] fibonacci(3) -> 2
[0.00039725s] fibonacci(4) -> 3
[0.00089025s] fibonacci(5) -> 5
[0.00123413s] fibonacci(6) -> 8
8


In [20]:
import functools
@functools.cache  #1: Add caching decorator
@clock  #2: cache is applied on the function returned by clock - stacked decs

def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)


if __name__ == '__main__':
    print(fibonacci(6))

[0.00000071s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00006958s] fibonacci(2) -> 1
[0.00000079s] fibonacci(3) -> 2
[0.00011325s] fibonacci(4) -> 3
[0.00000071s] fibonacci(5) -> 5
[0.00048213s] fibonacci(6) -> 8
8


In [30]:
# more extreme example -> let's remove the printing in clock
def fibonacci2(n):
    if n < 2:
        return n
    return fibonacci2(n - 2) + fibonacci2(n - 1)

# recursive
start = time.perf_counter()
fibonacci2(40)
end = time.perf_counter()
print(f"Recursion - Total time: {end - start:.2f}")

# using cache
@functools.cache
def fibonacci3(n):
    if n < 2:
        return n
    return fibonacci3(n - 2) + fibonacci3(n - 1)

# recursive
start = time.perf_counter()
fibonacci3(40)
end = time.perf_counter()
print(f"Caching - Total time: {end - start:.2f}")

Recursion - Total time: 20.56
Caching - Total time: 0.00


### stacked decorators:

```python
@alpha
@beta
def my_fn():
    ...
```
is equal to:

```python
my_fn = alpha(beta(my_fn))
```

### Parameterized Decorators

This gets a bit confusing since Python is going to do this in source:

- take decorated function and pass as first argument to decorator function

So, how would we get a parameter in? 
- Make a decorator factory...

In [31]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x108eeb040>)
running main()
registry -> [<function f1 at 0x108eeb040>]
running f1()


Now we update to determine if we want to actually register the decorated function. 

In [33]:
registry = set()  #1 Adding / Remove fast

def register(active=True):  #2 optional keyword argument now
    def decorate(func):  #3 This is our decorate
        print('running register'
              f'(active={active})->decorate({func})')
        if active:   # 4 add to registry if active
            registry.add(func)
        else:
            registry.discard(func)  # 5 discard if not active
            # and in registry 

        return func  # 6 -> must return a function
    return decorate  # 7 register is decorator factory

@register(active=False)  # 8: Won't be added
def f1():
    print('running f1()')

@register()  # 9: Will be added since active
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    
print(f"Contents: {registry}")

running register(active=False)->decorate(<function f1 at 0x108eebee0>)
running register(active=True)->decorate(<function f2 at 0x108eeb4c0>)
Contents: {<function f2 at 0x108eeb4c0>}
